In [202]:
import json
import pandas as pd
import numpy as np
from datetime import datetime

In [21]:
def read_axe_jsons(axe_num):
    with open('axe_listings/%s' % (axe_num), "r") as read_file:
        listing = json.load(read_file)
    with open('axe_specs/%s' % (axe_num), "r") as read_file:
        specs = json.load(read_file)
    return {'listing': listing, 'specs': specs}

In [132]:
read_axe_jsons("axe_372571263704.json")

{'listing': {'itemId': ['372571263704'],
  'title': ['GRETSCH G2655T STREAMLINER CENTER BLOCK JR. SEMI-HOLLOW BIGSBY BROAD TRON GUITAR'],
  'globalId': ['EBAY-US'],
  'primaryCategory': [{'categoryId': ['33034'],
    'categoryName': ['Electric Guitars']}],
  'galleryURL': ['http://thumbs1.ebaystatic.com/m/mRWeXEQsqSSkBgjzJ61xhzg/140.jpg'],
  'viewItemURL': ['http://www.ebay.com/itm/GRETSCH-G2655T-STREAMLINER-CENTER-BLOCK-JR-SEMI-HOLLOW-BIGSBY-BROAD-TRON-GUITAR-/372571263704'],
  'paymentMethod': ['PayPal'],
  'autoPay': ['false'],
  'postalCode': ['90601'],
  'location': ['Whittier,CA,USA'],
  'country': ['US'],
  'shippingInfo': [{'shippingType': ['Calculated'],
    'shipToLocations': ['Worldwide'],
    'expeditedShipping': ['true'],
    'oneDayShippingAvailable': ['false'],
    'handlingTime': ['1']}],
  'sellingStatus': [{'currentPrice': [{'@currencyId': 'USD',
      '__value__': '319.99'}],
    'convertedCurrentPrice': [{'@currencyId': 'USD', '__value__': '319.99'}],
    'bidCount'

In [493]:
class Axe:
    def __init__(self, file_name=None):
        self.id = file_name[4:-6]
        self.__body = read_axe_jsons("%s" % (file_name))
        self.price_usd = float(self.__body['listing']['sellingStatus'][0]['convertedCurrentPrice'][0]['__value__'])
        self.title = self.__body['listing']['title']
        self.market = self.__body['listing']['globalId'][0]
        self.pic = self.__body['listing']['galleryURL'][0]
        self.pic_big = self.__body['listing']['pictureURLLarge'][0]
        self.url = self.__body['listing']['viewItemURL'][0]
        self.pay_method = self.__body['listing']['paymentMethod'][0]
        self.autopay = self.__body['listing']['autoPay'][0]
        self.country_seller = self.__body['listing']['country'][0]
        self.ship_type = self.__body['listing']['shippingInfo'][0]['shippingType'][0]
        self.ship_worldwide = self.__body['listing']['shippingInfo'][0]['shipToLocations'][0] == 'Worldwide'
        self.ship_expedite = self.__body['listing']['shippingInfo'][0]['expeditedShipping'][0] == 'true'
        self.ship_one_day = self.__body['listing']['shippingInfo'][0]['oneDayShippingAvailable'][0] == 'true'
        self.ship_handling_time = int(self.__body['listing']['shippingInfo'][0]['handlingTime'][0])
        self.zip = self.__body['listing']['postalCode'][0]
        self.best_offer = self.__body['listing']['listingInfo'][0]['bestOfferEnabled'][0] == 'true'
        self.start_time = datetime.strptime(self.__body['listing']['listingInfo'][0]['startTime'][0], 
                                            "%Y-%m-%dT%H:%M:%S.%fZ")
        self.end_time = datetime.strptime(self.__body['listing']['listingInfo'][0]['endTime'][0], 
                                            "%Y-%m-%dT%H:%M:%S.%fZ")
        self.duration = int((self.end_time - self.start_time).days)
        self.returns = self.__body['listing']['returnsAccepted'][0] == 'true'
        self.multi = self.__body['listing']['isMultiVariationListing'][0] == 'true'
        self.top_rated = self.__body['listing']['topRatedListing'][0] == 'true'
        
        # End Generic Section // Begin Specs Section:
        
        self.best_offer = self.__body['specs']['BestOfferEnabled'] == True
        self.description = self.__body['specs']['Description']
        self.pic_quantity = len(self.__body['specs']['PictureURL'])
        self.seller_feedback_score = float(self.__body['specs']['Seller']['FeedbackScore'])
        self.seller_positive_percent = float(self.__body['specs']['Seller']['PositiveFeedbackPercent'])
        
        # Niche niche specifics:
        try:
            self.__attrs = {prop['Name']:prop['Value'][0] 
                            for prop in self.__body['specs']['ItemSpecifics']['NameValueList']}
            self.make = self.__attrs.get('Brand')
            self.model = self.__attrs.get('Model')
            if self.__attrs.get('Model Year'):
                self.year = int(self.__attrs.get('Model Year')[:4])
            self.material = self.__attrs.get('Body Material')
            self.right_left_hand = self.__attrs.get('Right-/ Left-Handed')
            self.country_manufacture = self.__attrs.get('Country/Region of Manufacture')
            self.body_type = self.__attrs.get('Body Type')
            self.string_config = self.__attrs.get('String Configuration')
            self.color = self.__attrs.get('Body Color')
        except KeyError:
            self.__attrs = []
            
        self.subtitle = self.__body['specs'].get('Subtitle')
        if self.returns:
            self.returns_time = int(self.__body['specs']['ReturnPolicy'].get('ReturnsWithin').split()[0])
        else: self.returns_time = None

In [502]:
axe = Axe('axe_163494046811.json')

In [503]:
axe.zip

'95372'

In [504]:
axe_113564436682.returns, axe_113564436682.returns_time

(False, None)

In [471]:
axe_113564436682._Axe__body

{'listing': {'itemId': ['143096563609'],
  'title': ['fender telecaster california series 97-98 serial no amxn713848 usa'],
  'globalId': ['EBAY-US'],
  'primaryCategory': [{'categoryId': ['33034'],
    'categoryName': ['Electric Guitars']}],
  'galleryURL': ['http://thumbs2.ebaystatic.com/m/mPWX6jhmfCKmQy_0z5tD3kQ/140.jpg'],
  'viewItemURL': ['http://www.ebay.com/itm/fender-telecaster-california-series-97-98-serial-no-amxn713848-usa-/143096563609'],
  'paymentMethod': ['PayPal'],
  'autoPay': ['false'],
  'postalCode': ['49508'],
  'location': ['Grand Rapids,MI,USA'],
  'country': ['US'],
  'shippingInfo': [{'shippingType': ['Calculated'],
    'shipToLocations': ['Worldwide'],
    'expeditedShipping': ['true'],
    'oneDayShippingAvailable': ['false'],
    'handlingTime': ['3']}],
  'sellingStatus': [{'currentPrice': [{'@currencyId': 'USD',
      '__value__': '509.99'}],
    'convertedCurrentPrice': [{'@currencyId': 'USD', '__value__': '509.99'}],
    'bidCount': ['2'],
    'sellingSt